In [1]:
import os

In [2]:
%pwd

'c:\\Users\\sharm\\Desktop\\DSProjects\\ML\\chicken_disease\\End-to-End-Chicken-Disease-Classification\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\sharm\\Desktop\\DSProjects\\ML\\chicken_disease\\End-to-End-Chicken-Disease-Classification'

In [7]:
# Entity

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
# Configuration manager
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [ ]:
# Data Ingestion Component
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [13]:
# pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-21 15:38:21,194: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-21 15:38:21,205: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-21 15:38:21,206: INFO: common: created directory at: artifacts]
[2025-07-21 15:38:21,206: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-21 15:38:22,752: INFO: 1170291011: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11617402
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "6e1b49cc9739ab6ca182b20431e0a4720f6bd6d5b88ebe3669b9a0840b9f652d"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6D36:348A08:94C6A:147557:687E50DC
Accept-Ranges: bytes
Date: Mon, 21 Jul 2025 14:38:22 GMT
Via: 1.1 varnish
X-Served-By: cache-lcy-eglc8600028-LCY
X-C